In [1]:
import pandas as pd
import numpy as np

In [2]:
# File to Load (Remember to Change These)
file_1 = "Homework\Instructions\HeroesOfPymoli\Resources\purchase_data.csv"

In [3]:
# Read Purchasing File and store into Pandas data frame
purchase_data = pd.read_csv(file_1)

In [4]:
#Find player count by finding unique screen names and finding the length of that list
player_count = len(purchase_data['SN'].unique())

# DataFrame creation for player count
players_df = pd.DataFrame([{'Total Players': player_count}])
#gets rid of number index and resets to Total Players 
players_df.set_index('Total Players', inplace = True)
players_df

""
Total Players
576


In [5]:
#code for inspecting data
#pur_data['Item ID'].value_counts()
total_unique = pd.DataFrame(purchase_data['Item ID'].unique())
len(total_unique)

#finds the number of total purchases by counting occurances of price
total_purchase = purchase_data['Price'].count()
#calculates total revenue for table by summing occurance of price and below calc
total_revenue = round(purchase_data['Price'].sum(),2)
#calculates total_rev
avg_price = round(total_revenue/total_purchase, 2)


In [6]:
#creates Purchase Analysis DataFrame
pur_analysis = pd.DataFrame([{
    
    "Number of Unique Items": total_unique,
    'Average Purchase Price': avg_price,
    'Total Purchases': total_purchase,
    'Total Revenue': total_revenue
}])

#format Purchases Analysis Table
pur_analysis.style.format({'Average Purchase Price': '${:.2f}', 'Total Revenue': '${:,.2f}'})

,Average Purchase Price,Number of Unique Items,Total Purchases,Total Revenue
0,$3.05,0 0 108 1 143 2 92 3 100 4 131 5 81 6 169 7 162 8 21 9 136 10 95 11 116 12 4 13 165 14 98 15 40 16 161 17 82 18 89 19 57 20 168 21 24 22 151 23 141 24 178 25 71 26 96 27 119 28 37 29 140 .. ... 153 38 154 91 155 118 156 67 157 45 158 49 159 121 160 79 161 14 162 43 163 31 164 64 165 86 166 93 167 26 168 122 169 61 170 48 171 112 172 126 173 115 174 55 175 132 176 127 177 47 178 130 179 90 180 177 181 104 182 180 [183 rows x 1 columns],780,"$2,379.77"


In [7]:
#creates df of unique player names by only keeping the last occurance
total_unique_players = purchase_data.drop_duplicates(['SN'], keep ='last')

#counts gender values from the df with no duplicate screen names
gender_counts = total_unique_players['Gender'].value_counts().reset_index()
#adds column for % of players using player count from first table and gender_count 
#column which is a count from line above
gender_counts['% of Players'] = gender_counts['Gender']/player_count * 100
#renames columns
gender_counts.rename(columns = {'index': 'Gender', 'Gender': '# of Players'}, inplace = True)
#sets index as Gender for aesthetics 
gender_counts.set_index(['Gender'], inplace = True)
#just checking percents sum to 100%
#gender_counts['% of Players'].sum()
#formats table
gender_counts.style.format({"% of Players": "{:.1f}%"})

,# of Players,% of Players
Gender,,
Male,484,84.0%
Female,81,14.1%
Other / Non-Disclosed,11,1.9%


In [8]:
# counts purchases by gender
purchase_count_by_gen = pd.DataFrame(purchase_data.groupby('Gender')['Gender'].count())
# sums price by gender
total_purchase_by_gen = pd.DataFrame(purchase_data.groupby('Gender')['Price'].sum())
#merges the two data frames from above
purchase_analysis_gen = pd.merge(purchase_count_by_gen, total_purchase_by_gen, left_index = True, right_index = True)
#renames columns
purchase_analysis_gen.rename(columns = {'Gender': '# of Purchases', 'Price':'Total Purchase Value'}, inplace=True)
#adds column for average purchase price by gender by dividing total purcahse value by gender by # of purchases by gender
purchase_analysis_gen['Average Purchase Price'] = purchase_analysis_gen['Total Purchase Value']/purchase_analysis_gen['# of Purchases']
#merges gender counts from above table (excluding dup SNs) into current df 
purchase_analysis_gen = purchase_analysis_gen.merge(gender_counts, left_index = True, right_index = True)
# calculates and adds normalized total column by dividing total purchase value by unique # of players by genger
purchase_analysis_gen['Normalized Totals'] = purchase_analysis_gen['Total Purchase Value']/purchase_analysis_gen['# of Players']
purchase_analysis_gen
#deletes columns not needed for table (# of Players was used for normalized totals while % of players came from gender count table)
del purchase_analysis_gen['% of Players']
del purchase_analysis_gen['# of Players']
# #resets index for aesthetics 
# # pur_analysis_gen.set_index('Gender', inplace=True)
# #formats table
purchase_analysis_gen.style.format({'Total Purchase Value': '${:.2f}', 'Average Purchase Price': '${:.2f}', 'Normalized Totals': '${:.2f}'})

,# of Purchases,Total Purchase Value,Average Purchase Price,Normalized Totals
Gender,,,,
Female,113,$361.94,$3.20,$4.47
Male,652,$1967.64,$3.02,$4.07
Other / Non-Disclosed,15,$50.19,$3.35,$4.56


In [9]:
#creates a column 'age_bin' based on conditional of age range
purchase_data.loc[(purchase_data['Age'] < 10), 'age_bin'] = "< 10"
purchase_data.loc[(purchase_data['Age'] >= 10) & (purchase_data['Age'] <= 14), 'age_bin'] = "10 - 14"
purchase_data.loc[(purchase_data['Age'] >= 15) & (purchase_data['Age'] <= 19), 'age_bin'] = "15 - 19"
purchase_data.loc[(purchase_data['Age'] >= 20) & (purchase_data['Age'] <= 24), 'age_bin'] = "20 - 24"
purchase_data.loc[(purchase_data['Age'] >= 25) & (purchase_data['Age'] <= 29), 'age_bin'] = "25 - 29"
purchase_data.loc[(purchase_data['Age'] >= 30) & (purchase_data['Age'] <= 34), 'age_bin'] = "30 - 34"
purchase_data.loc[(purchase_data['Age'] >= 35) & (purchase_data['Age'] <= 39), 'age_bin'] = "35 - 39"
purchase_data.loc[(purchase_data['Age'] >= 40), 'age_bin'] = "> 40"
#double checked count
# pur_data[['age_bin', 'Age']].count()

# counts purchases by age bin by counting screen names (non-unique)
purchase_count_age = pd.DataFrame(purchase_data.groupby('age_bin')['SN'].count())
#finds avg price of purchases by age bin
avg_price_age = pd.DataFrame(purchase_data.groupby('age_bin')['Price'].mean())
#finds total purchase value by age bin
total_purchase_age = pd.DataFrame(purchase_data.groupby('age_bin')['Price'].sum())
#deletes multiple occurances of SN while only keeping last, then counts # of unique
#players by age bin
total_unique_age = pd.DataFrame(purchase_data.drop_duplicates('SN', keep = 'last').groupby('age_bin')['SN'].count())
#merges all info from above into one df
merge_age = pd.merge(purchase_count_age, avg_price_age, left_index = True, right_index = True).merge(total_purchase_age, left_index = True, right_index = True).merge(total_unique_age, left_index = True, right_index = True)
#renames columns
merge_age.rename(columns = {"SN_x": "# of Purchases", "Price_x": "Average Purchase Price", "Price_y": "Total Purchase Value", "SN_y": "# of Purchasers"}, inplace = True)
#calculates normalized totals
merge_age['Normalized Totals'] = merge_age['Total Purchase Value']/merge_age['# of Purchasers']
#rest index for aesthetics
merge_age.index.rename("Age", inplace = True)
# formats
merge_age.style.format({'Average Purchase Price': '${:.2f}', 'Total Purchase Value': '${:.2f}', 'Normalized Totals': '${:.2f}'})

,# of Purchases,Average Purchase Price,Total Purchase Value,# of Purchasers,Normalized Totals
Age,,,,,
10 - 14,28,$2.96,$82.78,22,$3.76
15 - 19,136,$3.04,$412.89,107,$3.86
20 - 24,365,$3.05,$1114.06,258,$4.32
25 - 29,101,$2.90,$293.00,77,$3.81
30 - 34,73,$2.93,$214.00,52,$4.12
35 - 39,41,$3.60,$147.67,31,$4.76
< 10,23,$3.35,$77.13,17,$4.54
> 40,13,$2.94,$38.24,12,$3.19


In [10]:
# Identify the the top 5 spenders in the game by total purchase value, then list (in a table):
# SN
# Purchase Count
# Average Purchase Price
# Total Purchase Value

#Group by screen name to find, total purchase per person, number of purchases per person, and average price price per person
purchase_amt_by_SN = pd.DataFrame(purchase_data.groupby('SN')['Price'].sum())
num_purchase_by_SN = pd.DataFrame(purchase_data.groupby('SN')['Price'].count())
avg_purchase_by_SN = pd.DataFrame(purchase_data.groupby('SN')['Price'].mean())
# merge the above dfs
merged_top5 = pd.merge(purchase_amt_by_SN, num_purchase_by_SN, left_index = True, right_index = True).merge(avg_purchase_by_SN, left_index=True, right_index=True)
# rename columns
merged_top5.rename(columns = {'Price_x': 'Total Purchase Value', 'Price_y':'Purchase Count', 'Price':'Average Purchase Price'}, inplace = True)
# sort from highest purchase value to lowest
merged_top5.sort_values('Total Purchase Value', ascending = False, inplace=True)
# take top 5 only
merged_top5 = merged_top5.head()
# format
merged_top5.style.format({'Total Purchase Value': '${:.2f}', 'Average Purchase Price': '${:.2f}'})

,Total Purchase Value,Purchase Count,Average Purchase Price
SN,,,
Lisosia93,$18.96,5,$3.79
Idastidru52,$15.45,4,$3.86
Chamjask73,$13.83,3,$4.61
Iral74,$13.62,4,$3.40
Iskadarya95,$13.10,3,$4.37


In [11]:
# gets a count of each item by grouping by Item ID and counting the number of each IDs occurances
top5_items_ID = pd.DataFrame(purchase_data.groupby('Item ID')['Item ID'].count())
#sort from high to low total purchase count
top5_items_ID.sort_values('Item ID', ascending = False, inplace = True)
#keep the first 6 rows because there is a tie
top5_items_ID = top5_items_ID.iloc[0:6][:]
#find the total purchase value of each item
top5_items_total = pd.DataFrame(purchase_data.groupby('Item ID')['Price'].sum())
#merge purcahse count and total purcahse value 
top5_items = pd.merge(top5_items_ID, top5_items_total, left_index = True, right_index = True)
#drop duplicate items from original Df
total_unique_items = purchase_data.drop_duplicates(['Item ID'], keep = 'last')
# merge to get all other info from the top 6 using the no dup df
top5_merge_ID = pd.merge(top5_items, total_unique_items, left_index = True, right_on = 'Item ID')
#keep only neede columns
top5_merge_ID = top5_merge_ID[['Item ID', 'Item Name', 'Item ID_x', 'Price_y', 'Price_x']]
#reset index as item ID for aesthetics
top5_merge_ID.set_index(['Item ID'], inplace = True)
# rename columns
top5_merge_ID.rename(columns =  {'Item ID_x': 'Purchase Count', 'Price_y': 'Item Price', 'Price_x': 'Total Purchase Value'}, inplace=True)
#format
top5_merge_ID.style.format({'Item Price': '${:.2f}', 'Total Purchase Value': '${:.2f}'})

C:\Users\busyd\Anaconda3\lib\site-packages\ipykernel_launcher.py:11: FutureWarning: 'Item ID' is both an index level and a column label.
Defaulting to column, but this will raise an ambiguity error in a future version
  # This is added back by InteractiveShellApp.init_path()


,Item Name,Purchase Count,Item Price,Total Purchase Value
Item ID,,,,
178,"Oathbreaker, Last Hope of the Breaking Storm",12,$4.23,$50.76
145,Fiery Glass Crusader,9,$4.58,$41.22
108,"Extraction, Quickblade Of Trembling Hands",9,$3.53,$31.77
82,Nirvana,9,$4.90,$44.10
19,"Pursuit, Cudgel of Necromancy",8,$1.02,$8.16
103,Singed Scalpel,8,$4.35,$34.80


In [12]:
# find total purcahse value and sort by high to low
top5_profit = pd.DataFrame(purchase_data.groupby('Item ID')['Price'].sum())
top5_profit.sort_values('Price', ascending = False, inplace = True)
# only keep top 5
top5_profit = top5_profit.iloc[0:5][:]
#get item purchase count
purchase_count_profit = pd.DataFrame(purchase_data.groupby('Item ID')['Item ID'].count())

top5_profit = pd.merge(top5_profit, purchase_count_profit, left_index = True, right_index = True, how = 'left')
top5_merge_profit = pd.merge(top5_profit, total_unique_items, left_index = True, right_on = 'Item ID', how = 'left')
top5_merge_profit = top5_merge_profit[['Item ID', 'Item Name', 'Item ID_x', 'Price_y','Price_x']]
top5_merge_profit.set_index(['Item ID'], inplace=True)
top5_merge_profit.rename(columns = {'Item ID_x': 'Purchase Count', 'Price_y': 'Item Price', 'Price_x': 'Total Purchase Value'}, inplace = True)
top5_merge_profit.style.format({'Item Price': '${:.2f}', 'Total Purchase Value': '${:.2f}'})

,Item Name,Purchase Count,Item Price,Total Purchase Value
Item ID,,,,
178,"Oathbreaker, Last Hope of the Breaking Storm",12,$4.23,$50.76
82,Nirvana,9,$4.90,$44.10
145,Fiery Glass Crusader,9,$4.58,$41.22
92,Final Critic,8,$4.88,$39.04
103,Singed Scalpel,8,$4.35,$34.80
